# Getting vehicle miles traveled by month for a given state

Jake Kara
jkara@trendct.org
Dec. 12, 2016


#### Problem

Data here (https://www.fhwa.dot.gov/policyinformation/travel_monitoring/tvt.cfm) is only available in monthly spreadsheets. There are no annual figures at the state level. I even called the Federal Highway Administration's PR room and they said the data is not available annualized at the state level, so this scrape was necessary. 

If you just want national data, that's in the "historical" spreadsheet (https://www.fhwa.dot.gov/policyinformation/travel_monitoring/historicvmt.xlsx) going back to 1970!

#### Solution

1. Download all XLS/XLSX files from above url using requests and BeautifulSoup (/scrape/vmtscrape.py)
2. Parse files in reverse chronological order until the format changes too significantly to be interested in going back any further.

#### Status

I was able to parse data from 2004 with only three blank months in 2007, which I could have manually fixed by looking at the spreadsheets, but my colleague only needed data back through 2009 anyway, so this was enough.

#### Taking it further

To customize this script for your state, just replace "Connecticut" in the process_2016() function.

The process_2016() function processes all spreadsheets that are in the same format as the 2016 files, or are minimally different. I put in a few conditions to handle the files back to 2004.

For earlier years, I would recommend writing a process_2003() method, to process 2003 data and as far back as possible until you hit hickups.


In [1]:
import pandas as pd, numpy as np, os, re, datetime

In [2]:
def date_from_fname(clean):
    d = datetime.datetime.strptime(clean, "%Y %b" )
    return d
date_from_fname("2012 apr")

datetime.datetime(2012, 4, 1, 0, 0)

In [3]:
#Clean up names 
def clean_mo(fname):
#     print "cleaning " + fname
    if fname.startswith("_"):
        fname = fname[1:]
    trimmed = fname[:fname.index("_")].lower()
    trimmed = re.sub("tvt","",trimmed)
    split_ind = re.search(r"[a-z]",trimmed).start()
    ret = "20" + trimmed[:split_ind] + " " + trimmed[split_ind:]
    
    return date_from_fname(ret)

clean_mo('16martvt_16martvt.xls')

datetime.datetime(2016, 3, 1, 0, 0)

In [4]:
# Process everything that conforms to the same format as the 2016 data
# Seems to work at least back to to 2011
def process_2016(fname):
    #print "processing " + fname
    # Skip rows until we hit a row with 'CMILES' in it
    col = "CMILES"
    if "07" in fname:
        ret = pd.read_excel("scrape/data/" + fname,sheetname=5)
        col="Unnamed: 2"
    elif "06" in fname or "05" in fname or "04" in fname or "03" in fname:
        ret = pd.read_excel("scrape/data/" + fname,sheetname=5)
        col="Unnamed: 4"
    else:
        for i in range(5, 10):
            ret = pd.read_excel("scrape/data/" + fname,sheetname=5,skiprows=i)
            if "CMILES" in ret.columns:
                break

    try:
        return int(ret[ret[ret.columns[0]] == "Connecticut"][col])
    except:
        print "Error processing " + fname + " with col " + col
        return -1
        
filenames = os.listdir("scrape/data")

def try_16(yr_abbr):
    year16 = filter(lambda x: str(yr_abbr).lower() in x.lower(),
                    filenames)
    cmiles = {}
    for mt in year16:
        int_cmiles = process_2016(mt)
        #print mt + ": " + str(int_cmiles)
        cmiles[clean_mo(mt)] = int_cmiles

    return cmiles

cmiles = {}

# Before 2004, the spreadsheet is different enough that it's not worth going back any farther for me for today
for i in range(4, 17):
    cmiles[i] = try_16(str(i).zfill(2))


Error processing 07febtvt_07febtvt.xls with col Unnamed: 2
Error processing 07jantvt_07jantvt.xls with col Unnamed: 2
Error processing 07martvt_07martvt.xls with col Unnamed: 2


In [8]:
cmiles

{4: {datetime.datetime(2004, 1, 1, 0, 0): 2267,
  datetime.datetime(2004, 2, 1, 0, 0): 2196,
  datetime.datetime(2004, 3, 1, 0, 0): 2562,
  datetime.datetime(2004, 4, 1, 0, 0): 2633,
  datetime.datetime(2004, 5, 1, 0, 0): 2749,
  datetime.datetime(2004, 6, 1, 0, 0): 2701,
  datetime.datetime(2004, 7, 1, 0, 0): 2823,
  datetime.datetime(2004, 8, 1, 0, 0): 2846,
  datetime.datetime(2004, 9, 1, 0, 0): 2724,
  datetime.datetime(2004, 10, 1, 0, 0): 2848,
  datetime.datetime(2004, 11, 1, 0, 0): 2593,
  datetime.datetime(2004, 12, 1, 0, 0): 2667},
 5: {datetime.datetime(2005, 1, 1, 0, 0): 2336,
  datetime.datetime(2005, 2, 1, 0, 0): 2257,
  datetime.datetime(2005, 3, 1, 0, 0): 2566,
  datetime.datetime(2005, 4, 1, 0, 0): 2619,
  datetime.datetime(2005, 5, 1, 0, 0): 2820,
  datetime.datetime(2005, 6, 1, 0, 0): 2784,
  datetime.datetime(2005, 7, 1, 0, 0): 2807,
  datetime.datetime(2005, 8, 1, 0, 0): 2813,
  datetime.datetime(2005, 9, 1, 0, 0): 2585,
  datetime.datetime(2005, 10, 1, 0, 0): 2638,

In [9]:
# Examining some of the sheets that cause breaks so I can add special conditions above

#pd.read_excel("scrape/data/07aprtvt_07aprtvt.xls",sheetname=5)
#pd.read_excel("scrape/data/06aprtvt_06aprtvt.xls",sheetname=5)
#pd.read_excel("scrape/data/05aprtvt_05aprtvt.xls",sheetname=5)


In [18]:
def merge_all():
    frames = []
    for k in cmiles.keys():
        frames.append(pd.DataFrame(cmiles[k],index=[0]).transpose().sort_index())
    ret = pd.concat(frames).reset_index()
    ret.columns = "month","miles"
    return ret

# Output a monthly CSV

monthly = merge_all()
monthly.to_csv("output/miles_traveled_monthly.csv")
monthly

,month,miles
0,2004-01-01,2267
1,2004-02-01,2196
2,2004-03-01,2562
3,2004-04-01,2633
4,2004-05-01,2749
5,2004-06-01,2701
6,2004-07-01,2823
7,2004-08-01,2846
8,2004-09-01,2724
9,2004-10-01,2848


In [24]:
def yearly(df):
    ret = df.copy()
    ret["year"] = ret["month"].apply(lambda x: x.year)
    ret = ret.groupby("year").agg({"miles":np.sum})
    return ret
    
yearly(monthly)

,miles
year,
2004,31609
2005,31378
2006,31095
2007,24276
2008,31068
2009,31109
2010,32173
2011,31425
2012,30864
